In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import difflib
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

A Movie Recommendation Project

In [3]:
df = pd.read_csv(r"C:\Users\iyano\Downloads\movies.csv")
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

Choosing the criteria to choose the recommendation on

In [10]:
selection = ['genres', 'keywords', 'production_companies', 'title', 'cast', 'director']
print(selection)

['genres', 'keywords', 'production_companies', 'title', 'cast', 'director']


In [13]:
#Filling missing data for the selection to be used

for select in selection:
    df[select] = df[select].fillna('')

In [14]:
combination = pd.concat([df.genres, df.keywords, df.production_companies, df.title, df.cast, df.director], axis = 1)
combination

,genres,keywords,production_companies,title,cast,director
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",Avatar,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",Pirates of the Caribbean: At World's End,Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",Spectre,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",The Dark Knight Rises,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",John Carter,Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton
...,...,...,...,...,...,...
4798,Action Crime Thriller,united states\u2013mexico barrier legs arms pa...,"[{""name"": ""Columbia Pictures"", ""id"": 5}]",El Mariachi,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,Robert Rodriguez
4799,Comedy Romance,,[],Newlyweds,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,Edward Burns
4800,Comedy Drama Romance TV Movie,date love at first sight narration investigati...,"[{""name"": ""Front Street Pictures"", ""id"": 3958}...","Signed, Sealed, Delivered",Eric Mabius Kristin Booth Crystal Lowe Geoff G...,Scott Smith
4801,,,[],Shanghai Calling,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,Daniel Hsia


Converting the combination from text to numbers using TfidfVectorizer to facilitate the recommendation

In [17]:
combinations = df.genres + ' ' + df.keywords + ' ' + df.production_companies + ' ' + df.title + ' ' + df.cast + ' ' + df.director
combinations

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  [] Newlyweds Edward Burns Kerr...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      [] Shanghai Calling Daniel Henney Eliza Coup...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [19]:
vectorizer = TfidfVectorizer()

In [20]:
combinations_numeric = vectorizer.fit_transform(combinations)

In [22]:
print(combinations_numeric)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 162558 stored elements and shape (4803, 25562)>
  Coords	Values
  (0, 5225)	0.06713883987146686
  (0, 5300)	0.0775602975482567
  (0, 11204)	0.09495479294204098
  (0, 21083)	0.08838552614645974
  (0, 11381)	0.08853989808472335
  (0, 9374)	0.17373838414526785
  (0, 8655)	0.1898348116087896
  (0, 11927)	0.1518499021235849
  (0, 22016)	0.29110882856873793
  (0, 24720)	0.10851443479346622
  (0, 8842)	0.20907737355666348
  (0, 21889)	0.1785786344514795
  (0, 17737)	0.11941978281139623
  (0, 13916)	0.16211450427863563
  (0, 11418)	0.14333200364011756
  (0, 18735)	0.14807989534629556
  (0, 13740)	0.11946964567473284
  (0, 2319)	0.16867692829658887
  (0, 23678)	0.11201030722541767
  (0, 8249)	0.11015112571580656
  (0, 11732)	0.09656070463877495
  (0, 9098)	0.0966296764722559
  (0, 2436)	0.11298877498064591
  (0, 10423)	0.1378714330459605
  (0, 10828)	0.12939093876480362
  :	:
  (4802, 18224)	0.14568936319654968
  (4802, 10304)	0.1419

In [23]:
#finding the cosine similarity

similarity = cosine_similarity(combinations_numeric)
similarity

array([[1.        , 0.0365847 , 0.05185979, ..., 0.02138869, 0.        ,
        0.0208683 ],
       [0.0365847 , 1.        , 0.02791283, ..., 0.04289244, 0.        ,
        0.01179281],
       [0.05185979, 0.02791283, 1.        , ..., 0.01342769, 0.04881207,
        0.01057872],
       ...,
       [0.02138869, 0.04289244, 0.01342769, ..., 1.        , 0.        ,
        0.05432606],
       [0.        , 0.        , 0.04881207, ..., 0.        , 1.        ,
        0.        ],
       [0.0208683 , 0.01179281, 0.01057872, ..., 0.05432606, 0.        ,
        1.        ]])

In [24]:
#Inputing a movie name by user

movie_name = input('Enter a movie name: ')

Enter a movie name:  Iron man


In [26]:
movie_title = df['title'].tolist()
print(movie_title)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [27]:
find_movie = difflib.get_close_matches(movie_name, movie_title)
find_movie

['Iron Man', 'Iron Man 3', 'Iron Man 2']

In [28]:
#Extracting the first movie
found = find_movie[0]

In [31]:
#Finding the index number of the found movie so as to be able to apply cosine similarity on it

find_index = df[df['title'] == found]['index'].values[0]
print(find_index)

68


In [33]:
#Finding the similarity of the movie to the list available

similarity_rank = list(enumerate(similarity[find_index]))
print(similarity_rank)   

[(0, np.float64(0.03693428411427329)), (1, np.float64(0.04687365594937558)), (2, np.float64(0.018877268854150163)), (3, np.float64(0.013219188523668428)), (4, np.float64(0.037920288992793136)), (5, np.float64(0.09640842393933817)), (6, np.float64(0.04608833065063644)), (7, np.float64(0.3366280659959758)), (8, np.float64(0.011471674062459034)), (9, np.float64(0.06197512427425329)), (10, np.float64(0.06290768445344236)), (11, np.float64(0.01497742380639396)), (12, np.float64(0.032691204152314406)), (13, np.float64(0.018209709702832692)), (14, np.float64(0.10471798746630295)), (15, np.float64(0.02671678157531482)), (16, np.float64(0.37306534642585726)), (17, np.float64(0.016976594963357603)), (18, np.float64(0.02717206564226744)), (19, np.float64(0.04956079958567566)), (20, np.float64(0.17352703267876396)), (21, np.float64(0.01906577450611035)), (22, np.float64(0.013755338303700404)), (23, np.float64(0.013660286278371266)), (24, np.float64(0.018553657998849102)), (25, np.float64(0.0063087

In [34]:
#Sort the rank in descending order

descending_rank = sorted(similarity_rank, key = lambda x:x[1], reverse = True)
print(descending_rank)

[(68, np.float64(1.0000000000000002)), (79, np.float64(0.6446486396652629)), (31, np.float64(0.534252140670409)), (16, np.float64(0.37306534642585726)), (7, np.float64(0.3366280659959758)), (182, np.float64(0.32377801248373717)), (85, np.float64(0.3052218488986125)), (26, np.float64(0.27991219295868747)), (126, np.float64(0.27153011367276975)), (169, np.float64(0.2420055606992468)), (174, np.float64(0.24125307800563084)), (129, np.float64(0.23403837201118766)), (94, np.float64(0.21326390973160503)), (511, np.float64(0.20230777983971124)), (203, np.float64(0.19686906445642288)), (64, np.float64(0.1950655866560477)), (788, np.float64(0.19010383034510597)), (46, np.float64(0.18608354233146218)), (38, np.float64(0.18331802842029696)), (30, np.float64(0.1788974782804514)), (20, np.float64(0.17352703267876396)), (101, np.float64(0.1735198050374317)), (3623, np.float64(0.1612278498513221)), (232, np.float64(0.1584921980903676)), (1740, np.float64(0.1583643160506773)), (805, np.float64(0.15636

In [46]:
#First 20 recommended movie

i = 1

for number in descending_rank:
    index = number[0]
    recommended_title = df[df['index'] == index]['title'].values[0]
    if i < 21:
        print(i, '.' , recommended_title)

    i += 1

1 . Iron Man
2 . Iron Man 2
3 . Iron Man 3
4 . The Avengers
5 . Avengers: Age of Ultron
6 . Ant-Man
7 . Captain America: The Winter Soldier
8 . Captain America: Civil War
9 . Thor: The Dark World
10 . Captain America: The First Avenger
11 . The Incredible Hulk
12 . Thor
13 . Guardians of the Galaxy
14 . X-Men
15 . X2
16 . X-Men: Apocalypse
17 . Deadpool
18 . X-Men: Days of Future Past
19 . The Amazing Spider-Man 2
20 . Spider-Man 2


Compilation of the code for movie recommendation

In [47]:
movie_name = input('Enter a movie name: ')

find_movie = difflib.get_close_matches(movie_name, movie_title)

found = find_movie[0]

find_index = df[df['title'] == found]['index'].values[0]

similarity_rank = list(enumerate(similarity[find_index]))

descending_rank = sorted(similarity_rank, key = lambda x:x[1], reverse = True)

i = 1

for number in descending_rank:
    index = number[0]
    recommended_title = df[df['index'] == index]['title'].values[0]
    if i < 21:
        print(i, '.' , recommended_title)

    i += 1

Enter a movie name:  Super Man


1 . Superman
2 . Superman II
3 . Superman IV: The Quest for Peace
4 . Superman III
5 . Superman Returns
6 . Suicide Squad
7 . The Dark Knight
8 . Batman Begins
9 . Batman v Superman: Dawn of Justice
10 . Man of Steel
11 . The Dark Knight Rises
12 . Batman Returns
13 . Crimson Tide
14 . Batman
15 . Lethal Weapon 4
16 . Watchmen
17 . Conspiracy Theory
18 . Green Lantern
19 . Assassins
20 . Steel
